## 1. Whatsuplulu - Get Twitter data

In [ ]:
import twitter, re, datetime, pandas as pd

twitter_keys = {
    'consumer_key':        '3Fj7Pe0KT2ykaMqZO30bdur60',
    'consumer_secret':     'IoIrOGZg8n4bXPVb9KWTlKnQSCnb9csXjPwGTjmFHSb8zaqt0w',
    'access_token_key':    '60872859-5wHCWu9NOF29yX5AgosKPs6ZAebbAZH4ptAQTFhOq',
    'access_token_secret': 'jjS7bK5DE2a2fFu5R8wafbBVURiStn8DB1aZRoDLoBUBP'
}

api = twitter.Api(
    consumer_key         =   twitter_keys['consumer_key'],
    consumer_secret      =   twitter_keys['consumer_secret'],
    access_token_key     =   twitter_keys['access_token_key'],
    access_token_secret  =   twitter_keys['access_token_secret']
)


In [ ]:
class TweetMiner(object):

    result_limit    =   20    
    api             =   False
    data            =   []
    
    twitter_keys = {
        'consumer_key':        '3Fj7Pe0KT2ykaMqZO30bdur60',
        'consumer_secret':     'IoIrOGZg8n4bXPVb9KWTlKnQSCnb9csXjPwGTjmFHSb8zaqt0w',
        'access_token_key':    '60872859-5wHCWu9NOF29yX5AgosKPs6ZAebbAZH4ptAQTFhOq',
        'access_token_secret': 'jjS7bK5DE2a2fFu5R8wafbBVURiStn8DB1aZRoDLoBUBP'
    }
    
    def __init__(self, keys_dict, api, result_limit = 20):
        self.api = api
        self.twitter_keys = keys_dict
        self.result_limit = result_limit

    def mine_user_tweets(self, user="roshanlulu", mine_rewteets=False, max_pages=5):
        data           =  []
        last_tweet_id  =  False
        page           =  1
        
        while page <= max_pages:
            
            if last_tweet_id:
                statuses   =   self.api.GetUserTimeline(screen_name=user, count=self.result_limit, max_id=last_tweet_id - 1)        
            else:
                statuses   =   self.api.GetUserTimeline(screen_name=user, count=self.result_limit)
                
            for item in statuses:

                mined = {
                    'tweet_id':        item.id,
                    'handle':          item.user.name,
                    'retweet_count':   item.retweet_count,
                    'text':            item.text,
                    'mined_at':        datetime.datetime.now(),
                    'created_at':      item.created_at,
                }
                
                last_tweet_id = item.id
                data.append(mined)
                
            page += 1
            
        return data

In [ ]:
miner = TweetMiner(twitter_keys, api, result_limit=2)

In [ ]:
redbubble = miner.mine_user_tweets(user="redbubble", max_pages=100)

In [ ]:
print(redbubble)

In [ ]:
redbubble_df = pd.DataFrame(redbubble)

In [ ]:
redbubble_df.shape

In [ ]:
redbubble_df.head()

## Any interesting n-grams?

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter

# We can use the TfidfVectorizer to find ngrams for us
vect = TfidfVectorizer(ngram_range=(2,4))

# Pulls all of trumps tweet text's into one giant string
summaries = "".join(redbubble_df['text'])
ngrams_summaries = vect.build_analyzer()(summaries)

Counter(ngrams_summaries).most_common(20)

In [ ]:
tweets = redbubble_df

In [ ]:
# Using the textacy package to do some more comprehensive preprocessing
# http://textacy.readthedocs.io/en/latest/
from textacy.preprocess import preprocess_text

tweet_text = tweets['text'].values
clean_text = [preprocess_text(x, fix_unicode=True, lowercase=True, transliterate=False,
                              no_urls=True, no_emails=True, no_phone_numbers=True, no_currency_symbols=True,
                              no_punct=True, no_accents=True)
              for x in tweet_text]

In [ ]:
print(tweet_text[0:3])

In [ ]:
print(clean_text[0:3])

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# What is the average re-tweet count
tweets.retweet_count.describe()

In [ ]:
sns.boxplot(x=tweets['retweet_count'])
plt.show()

In [ ]:
# target is the handle.
# make trump 1 and sanders 0
y = tweets['retweet_count'].map(lambda x: 1 if x > 10 else 0).values

In [ ]:
print(np.mean(y))

In [ ]:
tweets['retweet'] = y

In [ ]:
tweets[tweets['retweet'] == 1].head()

In [ ]:
from sklearn.linear_model import LogisticRegression

# Preprocess our text data to Tfidf
tfv = TfidfVectorizer(ngram_range=(1,4), max_features=1000)
X = tfv.fit_transform(clean_text).todense()
print(X.shape)

In [ ]:
# cross-validate the accuracy:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(LogisticRegression(), X, y, cv=2)

import numpy as np
print (accuracies)
print (np.mean(accuracies))

# Setup logistic regression (or try another classification method here)
estimator = LogisticRegression()
estimator.fit(X, y)

## 2. Write to twitter api

In [9]:
import tweepy
import pandas as pd
consumer_key=        '3Fj7Pe0KT2ykaMqZO30bdur60'
consumer_secret=     'IoIrOGZg8n4bXPVb9KWTlKnQSCnb9csXjPwGTjmFHSb8zaqt0w'
access_token_key=    '60872859-5wHCWu9NOF29yX5AgosKPs6ZAebbAZH4ptAQTFhOq'
access_token_secret= 'jjS7bK5DE2a2fFu5R8wafbBVURiStn8DB1aZRoDLoBUBP'
        
# Set up OAuth and integrate with API
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token_key, access_token_secret)
api = tweepy.API(auth)



In [11]:
text = []

In [10]:
def post_tweet(msg):
    print(api.me().name)
    api.update_status(msg)
#     api.update_with_media('cloud.png', status = msg)

In [4]:
# Write a tweet to push to our Twitter account
post_tweet('Hello twitter, I am writing to you using @tweepy')

roshan lulu


In [23]:
#override tweepy.StreamListener to add logic to on_status
class MyStreamListener(tweepy.StreamListener):

    def on_status(self, status):
        print(status.user)
        text.append(status.text)
#     def on_error(self, status_code):
#         if status_code == 420:
#             #returning False in on_data disconnects the stream
#             return False
#     def disconnect(self):
#         if self.running is False:
#             return
#         self.running = False

In [25]:
myStreamListener = MyStreamListener()
myStream = tweepy.Stream(auth = api.auth, listener=myStreamListener)

In [26]:
myStream.filter(track=['G20Summit'])

User(_api=<tweepy.api.API object at 0x11177dfd0>, _json={'id': 2374450009, 'id_str': '2374450009', 'name': 'Gwyneth B 🗽🇺🇸🗽🙏', 'screen_name': 'gbellflowers50', 'location': 'Vancouver, WA', 'url': None, 'description': 'Wife! Mother! TRUE PATRIOT! 🇺🇸 🇺🇸 I RESIST! GOP! Donald Trump!  And His Partner In Crime! Mike Pence! I Will Fight Against Everything Both Stand For!', 'protected': False, 'verified': False, 'followers_count': 806, 'friends_count': 714, 'listed_count': 8, 'favourites_count': 48397, 'statuses_count': 50165, 'created_at': 'Wed Mar 05 22:47:15 +0000 2014', 'utc_offset': None, 'time_zone': None, 'geo_enabled': True, 'lang': 'en', 'contributors_enabled': False, 'is_translator': False, 'profile_background_color': 'C0DEED', 'profile_background_image_url': 'http://abs.twimg.com/images/themes/theme1/bg.png', 'profile_background_image_url_https': 'https://abs.twimg.com/images/themes/theme1/bg.png', 'profile_background_tile': False, 'profile_link_color': '1DA1F2', 'profile_sidebar_bo

User(_api=<tweepy.api.API object at 0x11177dfd0>, _json={'id': 804025594723835904, 'id_str': '804025594723835904', 'name': 'Ron Bellus', 'screen_name': 'RonBellus', 'location': None, 'url': None, 'description': None, 'protected': False, 'verified': False, 'followers_count': 12, 'friends_count': 22, 'listed_count': 0, 'favourites_count': 4496, 'statuses_count': 1061, 'created_at': 'Wed Nov 30 18:13:34 +0000 2016', 'utc_offset': None, 'time_zone': None, 'geo_enabled': False, 'lang': 'en', 'contributors_enabled': False, 'is_translator': False, 'profile_background_color': 'F5F8FA', 'profile_background_image_url': '', 'profile_background_image_url_https': '', 'profile_background_tile': False, 'profile_link_color': '1DA1F2', 'profile_sidebar_border_color': 'C0DEED', 'profile_sidebar_fill_color': 'DDEEF6', 'profile_text_color': '333333', 'profile_use_background_image': True, 'profile_image_url': 'http://abs.twimg.com/sticky/default_profile_images/default_profile_normal.png', 'profile_image_ur

User(_api=<tweepy.api.API object at 0x11177dfd0>, _json={'id': 264998118, 'id_str': '264998118', 'name': '🇺🇸 #MAGA 🇺🇸', 'screen_name': 'MunichEleven', 'location': None, 'url': None, 'description': 'Support our @POTUS @RealDonaldTrump in his mission to  #MAGA | #1A #2A | #Borders l #AmericaFirst | Pro-Israel 🇺🇸🇮🇱 Follow our national treasure @realJamesWoods', 'protected': False, 'verified': False, 'followers_count': 1776, 'friends_count': 948, 'listed_count': 51, 'favourites_count': 106, 'statuses_count': 26181, 'created_at': 'Sat Mar 12 20:48:54 +0000 2011', 'utc_offset': -25200, 'time_zone': 'Pacific Time (US & Canada)', 'geo_enabled': False, 'lang': 'en', 'contributors_enabled': False, 'is_translator': False, 'profile_background_color': 'C0DEED', 'profile_background_image_url': 'http://pbs.twimg.com/profile_background_images/490235172/bar_munich_11_olympians.jpg', 'profile_background_image_url_https': 'https://pbs.twimg.com/profile_background_images/490235172/bar_munich_11_olympians.

User(_api=<tweepy.api.API object at 0x11177dfd0>, _json={'id': 796539851843764224, 'id_str': '796539851843764224', 'name': 'Ana Banana', 'screen_name': 'ana_luvs_TWD', 'location': 'A place without those trumps', 'url': None, 'description': '#TheResistance #ProChoice #LGBTQ #ClimateChange #FollowBackResistance Trump was born through the butthole of a troll in the swamplands of Mordor.    ❤️ #TWD 😍', 'protected': False, 'verified': False, 'followers_count': 1423, 'friends_count': 1552, 'listed_count': 4, 'favourites_count': 7078, 'statuses_count': 11523, 'created_at': 'Thu Nov 10 02:27:54 +0000 2016', 'utc_offset': None, 'time_zone': None, 'geo_enabled': False, 'lang': 'en', 'contributors_enabled': False, 'is_translator': False, 'profile_background_color': 'F5F8FA', 'profile_background_image_url': '', 'profile_background_image_url_https': '', 'profile_background_tile': False, 'profile_link_color': '1DA1F2', 'profile_sidebar_border_color': 'C0DEED', 'profile_sidebar_fill_color': 'DDEEF6',

User(_api=<tweepy.api.API object at 0x11177dfd0>, _json={'id': 775374224034783232, 'id_str': '775374224034783232', 'name': 'Emperor_Palpitations', 'screen_name': 'RealAdamBomb78', 'location': 'Michigan, USA', 'url': 'http://google.com', 'description': 'Using our unlocked human cerebral plasticity for the greater good.\n\nFree Digital Media Quotes:\n@jdam_grafx\nadambombgrafx@gmail.com', 'protected': False, 'verified': False, 'followers_count': 1394, 'friends_count': 1434, 'listed_count': 73, 'favourites_count': 40842, 'statuses_count': 55600, 'created_at': 'Mon Sep 12 16:43:15 +0000 2016', 'utc_offset': -25200, 'time_zone': 'Pacific Time (US & Canada)', 'geo_enabled': False, 'lang': 'en', 'contributors_enabled': False, 'is_translator': False, 'profile_background_color': '000000', 'profile_background_image_url': 'http://abs.twimg.com/images/themes/theme1/bg.png', 'profile_background_image_url_https': 'https://abs.twimg.com/images/themes/theme1/bg.png', 'profile_background_tile': False, 

...

User(_api=<tweepy.api.API object at 0x11177dfd0>, _json={'id': 3249282368, 'id_str': '3249282368', 'name': 'SMorgan', 'screen_name': 'Shezz1511', 'location': None, 'url': None, 'description': 'conservative, business owner, Mom of 2 great kids!', 'protected': False, 'verified': False, 'followers_count': 1089, 'friends_count': 1365, 'listed_count': 31, 'favourites_count': 12095, 'statuses_count': 14714, 'created_at': 'Fri Jun 19 00:15:02 +0000 2015', 'utc_offset': None, 'time_zone': None, 'geo_enabled': False, 'lang': 'en', 'contributors_enabled': False, 'is_translator': False, 'profile_background_color': 'C0DEED', 'profile_background_image_url': 'http://abs.twimg.com/images/themes/theme1/bg.png', 'profile_background_image_url_https': 'https://abs.twimg.com/images/themes/theme1/bg.png', 'profile_background_tile': False, 'profile_link_color': '1DA1F2', 'profile_sidebar_border_color': 'C0DEED', 'profile_sidebar_fill_color': 'DDEEF6', 'profile_text_color': '333333', 'profile_use_background_

User(_api=<tweepy.api.API object at 0x11177dfd0>, _json={'id': 438990599, 'id_str': '438990599', 'name': 'Manumit WakeUp World', 'screen_name': 'Manumit1963', 'location': 'everywhere', 'url': 'http://bit.ly/2kGCfjb', 'description': 'free from slavery, servitude, emancipate freedom to think and do and not be told #Wakeup #Bitcoin   #Cyrptocurrency #Deplorable https://youtu.be/GpLAzAZXi2A #FB', 'protected': False, 'verified': False, 'followers_count': 1702, 'friends_count': 4868, 'listed_count': 44, 'favourites_count': 4193, 'statuses_count': 17360, 'created_at': 'Sat Dec 17 07:24:42 +0000 2011', 'utc_offset': None, 'time_zone': None, 'geo_enabled': False, 'lang': 'en', 'contributors_enabled': False, 'is_translator': False, 'profile_background_color': 'C0DEED', 'profile_background_image_url': 'http://abs.twimg.com/images/themes/theme1/bg.png', 'profile_background_image_url_https': 'https://abs.twimg.com/images/themes/theme1/bg.png', 'profile_background_tile': False, 'profile_link_color':

User(_api=<tweepy.api.API object at 0x11177dfd0>, _json={'id': 27686745, 'id_str': '27686745', 'name': 'America Matters!🇺🇸', 'screen_name': 'Collette_AZ', 'location': None, 'url': None, 'description': 'Live, Love, Laugh - I block those selling by spam.  Republican (the new) GOP.  Never a GOPe! May Elitists Rot in Hell!', 'protected': False, 'verified': False, 'followers_count': 3076, 'friends_count': 760, 'listed_count': 202, 'favourites_count': 79140, 'statuses_count': 114032, 'created_at': 'Mon Mar 30 17:31:54 +0000 2009', 'utc_offset': -25200, 'time_zone': 'America/Phoenix', 'geo_enabled': False, 'lang': 'en', 'contributors_enabled': False, 'is_translator': False, 'profile_background_color': '820000', 'profile_background_image_url': 'http://pbs.twimg.com/profile_background_images/79746108/x80c5bab9cb735d93c526265c3c83f7d.png', 'profile_background_image_url_https': 'https://pbs.twimg.com/profile_background_images/79746108/x80c5bab9cb735d93c526265c3c83f7d.png', 'profile_background_til

User(_api=<tweepy.api.API object at 0x11177dfd0>, _json={'id': 850499575789928448, 'id_str': '850499575789928448', 'name': 'Peppergirl', 'screen_name': 'pintsizedfemale', 'location': 'Minnesota, USA', 'url': None, 'description': "Trump is my President. Christian, love my husband, 29 years💕, 2A, CCP,  I support our Military, LEO's, & 1st resp. No lists or groups. I block stupidity. 😕", 'protected': False, 'verified': False, 'followers_count': 1045, 'friends_count': 1217, 'listed_count': 2, 'favourites_count': 12580, 'statuses_count': 15662, 'created_at': 'Sat Apr 08 00:04:35 +0000 2017', 'utc_offset': None, 'time_zone': None, 'geo_enabled': False, 'lang': 'en', 'contributors_enabled': False, 'is_translator': False, 'profile_background_color': 'F5F8FA', 'profile_background_image_url': '', 'profile_background_image_url_https': '', 'profile_background_tile': False, 'profile_link_color': '1DA1F2', 'profile_sidebar_border_color': 'C0DEED', 'profile_sidebar_fill_color': 'DDEEF6', 'profile_tex

KeyboardInterrupt: 

In [ ]:
del myStream

In [19]:
text[0]

'RT @RepTedLieu: Dear @POTUS, we understand you don\'t like to read &amp; instead prefer "killer graphics." Hopefully these pictures help… '